## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=2)

# Get the date one week ago
one_week_ago = today - timedelta(days=8)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

June 6 June 14


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,June-9-2025,Condo/Co-op,4242 Deste Ct #305,Lake Worth,FL,33467.0,134999.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Greenacres/4242-Dest...,PBBOR,25-821,N,Y,26.612095,-80.149542


In [8]:
print('Input county name:')
county = input()

Input county name:


 Palm Beach


In [9]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [12]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-06-09    27
2025-06-12    22
2025-06-13    16
2025-06-11    14
2025-06-10    13
Name: count, dtype: int64

In [13]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [14]:
df_top_ten = df2.head(10)

In [16]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
72,PAST SALE,2025-06-13,Condo/Co-op,1100 S Flagler Dr #1502,West Palm Beach,FL,33401.0,12543000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-11029308,N,Y,26.703917,-80.049385
3,PAST SALE,2025-06-11,Condo/Co-op,220 SE Mizner Blvd #303,Boca Raton,FL,33432.0,2753000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11091056,N,Y,26.347042,-80.083200
17,PAST SALE,2025-06-11,Condo/Co-op,184 Lake Dr Unit 1-201,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Shores/18...,Beaches MLS,RX-11076265,N,Y,26.781054,-80.039562
2,PAST SALE,2025-06-13,Condo/Co-op,220 SE Mizner Blvd #102,Boca Raton,FL,33432.0,2039000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11090640,N,Y,26.347042,-80.083200
86,PAST SALE,2025-06-13,Condo/Co-op,200 E Palmetto Park Rd #409,Boca Raton,FL,33432.0,1875000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Beaches MLS,RX-11030421,N,Y,26.350338,-80.082988
88,PAST SALE,2025-06-10,Condo/Co-op,170 N Ocean Blvd #307,Palm Beach,FL,33480.0,1690800.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Beaches MLS,RX-11013110,N,Y,26.721061,-80.035205
75,PAST SALE,2025-06-09,Condo/Co-op,4000 N Ocean Dr #301,Singer Island,FL,33404.0,1650000.0,2.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4000-N...,Beaches MLS,RX-11053747,N,Y,26.795862,-80.032351
77,PAST SALE,2025-06-13,Condo/Co-op,2800 S Ocean Blvd S Unit 19 C,Boca Raton,FL,33432.0,1635700.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Beaches MLS,RX-11045437,N,Y,26.323476,-80.075019
12,PAST SALE,2025-06-10,Condo/Co-op,4301 N Ocean Blvd #201,Boca Raton,FL,33431.0,1387500.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Beaches MLS,RX-11069723,N,Y,26.389260,-80.067914
25,PAST SALE,2025-06-09,Condo/Co-op,19980 Sawgrass Ln #5104,Boca Raton,FL,33434.0,1200000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/19980-Saw...,Beaches MLS,RX-11068508,N,Y,26.385217,-80.155402


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [17]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [18]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [19]:
response_df = pd.DataFrame(response_list)

In [20]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [21]:
df_top_ten = merged_df

In [22]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-06-13,Condo/Co-op,1100 S Flagler Dr #1502,West Palm Beach,FL,33401.0,12543000.0,3.0,4.5,...,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Christian Angle,Christian Angle Real Estate,None,None,Julien Taylor IV,William Raveis Real Estate,None,None
1,PAST SALE,2025-06-11,Condo/Co-op,220 SE Mizner Blvd #303,Boca Raton,FL,33432.0,2753000.0,2.0,3.0,...,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Alejandro Salazar,Douglas Elliman,Non Non-Member,Non-Member Selling Office,None,None
2,PAST SALE,2025-06-11,Condo/Co-op,184 Lake Dr Unit 1-201,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,...,-80.039562,https://www.redfin.com/FL/Palm-Beach-Shores/18...,Thomas Murray,Compass Florida LLC (PB),None,None,Thomas Murray,Compass Florida LLC (PB),None,None
3,PAST SALE,2025-06-13,Condo/Co-op,220 SE Mizner Blvd #102,Boca Raton,FL,33432.0,2039000.0,2.0,2.5,...,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Alejandro Salazar,Douglas Elliman,Non Agent,Non-Member Selling Office,None,None
4,PAST SALE,2025-06-13,Condo/Co-op,200 E Palmetto Park Rd #409,Boca Raton,FL,33432.0,1875000.0,2.0,2.5,...,-80.082988,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Scott Eckert,RPE Realty,None,None
5,PAST SALE,2025-06-10,Condo/Co-op,170 N Ocean Blvd #307,Palm Beach,FL,33480.0,1690800.0,2.0,2.0,...,-80.035205,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Karen Lehrman Prayias,"Sotheby's Intl. Realty, Inc.",None,None,Karen Lehrman Prayias,"Sotheby's Intl. Realty, Inc.",None,None
6,PAST SALE,2025-06-09,Condo/Co-op,4000 N Ocean Dr #301,Singer Island,FL,33404.0,1650000.0,2.0,3.5,...,-80.032351,https://www.redfin.com/FL/Riviera-Beach/4000-N...,Emmy Buckman,Illustrated Properties LLC (Co,Owen Buckman,Illustrated Properties LLC (Co,Meike MacGregor,Compass Florida LLC,None,None
7,PAST SALE,2025-06-13,Condo/Co-op,2800 S Ocean Blvd S Unit 19 C,Boca Raton,FL,33432.0,1635700.0,2.0,2.0,...,-80.075019,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Andrew Pietrak,Luxury Real Estate Group LLC,None,None,Robert Goodman,LoKation,None,None
8,PAST SALE,2025-06-10,Condo/Co-op,4301 N Ocean Blvd #201,Boca Raton,FL,33431.0,1387500.0,3.0,3.0,...,-80.067914,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Lisa Nanfra,Compass Florida LLC,Beth Aronson Fisher,Compass Florida LLC,Sian Matthew,Coldwell Banker,None,None
9,PAST SALE,2025-06-09,Condo/Co-op,19980 Sawgrass Ln #5104,Boca Raton,FL,33434.0,1200000.0,3.0,2.0,...,-80.155402,https://www.redfin.com/FL/Boca-Raton/19980-Saw...,Abigail Wapner,Abigail Wapner Realty,Barbara Orenstein,Abigail Wapner Realty,Ina Tuchman,Boca West Realty LLC,None,None


## Current Week's Values

In [23]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

92


In [24]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$598,087


In [25]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$353


In [26]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$55,024,048


In [27]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           92              598087          353                     55024048.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [28]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
91
-----------
Input Previous Week Condo Average Sales Price:
592601
-----------
Input Previous Week Condo Average PSF:
341
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
53926699.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [29]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [30]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [31]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [32]:
pd.set_option('display.max_columns',None)

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-13,Condo/Co-op,1100 S Flagler Dr #1502,West Palm Beach,FL,33401.0,12543000.0,3.0,4.5,Bristol Condo,3815.0,NaN,2019.0,NaN,3288.0,8327.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-11029308,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Christian Angle,Christian Angle Real Estate,None,None,Julien Taylor IV,William Raveis Real Estate,None,None,1,orange


In [34]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [35]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-13,Condo/Co-op,1100 S Flagler Dr #1502,West Palm Beach,FL,33401.0,12543000.0,3.0,4.5,Bristol Condo,3815.0,NaN,2019.0,NaN,3288.0,8327.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-11029308,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Christian Angle,Christian Angle Real Estate,None,None,Julien Taylor IV,William Raveis Real Estate,None,None,1,orange
1,PAST SALE,2025-06-11,Condo/Co-op,220 SE Mizner Blvd #303,Boca Raton,FL,33432.0,2753000.0,2.0,3.0,Alina Residences,2701.0,180.0,2024.0,NaN,1019.0,2283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11091056,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Alejandro Salazar,Douglas Elliman,Non Non-Member,Non-Member Selling Office,None,None,2,blue
2,PAST SALE,2025-06-11,Condo/Co-op,184 Lake Dr Unit 1-201,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,Havn Yacht Club & Residences,2662.0,NaN,2008.0,NaN,939.0,3993.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Shores/18...,Beaches MLS,RX-11076265,N,Y,26.781054,-80.039562,https://www.redfin.com/FL/Palm-Beach-Shores/18...,Thomas Murray,Compass Florida LLC (PB),None,None,Thomas Murray,Compass Florida LLC (PB),None,None,3,blue
3,PAST SALE,2025-06-13,Condo/Co-op,220 SE Mizner Blvd #102,Boca Raton,FL,33432.0,2039000.0,2.0,2.5,Alina Residences,1937.0,180.0,2024.0,NaN,1053.0,1625.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11090640,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Alejandro Salazar,Douglas Elliman,Non Agent,Non-Member Selling Office,None,None,4,blue
4,PAST SALE,2025-06-13,Condo/Co-op,200 E Palmetto Park Rd #409,Boca Raton,FL,33432.0,1875000.0,2.0,2.5,200 East Palmetto Park Condo,2018.0,2018.0,2009.0,NaN,929.0,2292.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Beaches MLS,RX-11030421,N,Y,26.350338,-80.082988,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Scott Eckert,RPE Realty,None,None,5,blue
5,PAST SALE,2025-06-10,Condo/Co-op,170 N Ocean Blvd #307,Palm Beach,FL,33480.0,1690800.0,2.0,2.0,Ocean Towers North Condo,1395.0,NaN,1968.0,NaN,1212.0,2138.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Beaches MLS,RX-11013110,N,Y,26.721061,-80.035205,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Karen Lehrman Prayias,"Sotheby's Intl. Realty, Inc.",None,None,Karen Lehrman Prayias,"Sotheby's Intl. Realty, Inc.",None,None,6,blue
6,PAST SALE,2025-06-09,Condo/Co-op,4000 N Ocean Dr #301,Singer Island,FL,33404.0,1650000.0,2.0,3.5,Martinique,2011.0,NaN,1986.0,NaN,820.0,1901.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4000-N...,Beaches MLS,RX-11053747,N,Y,26.795862,-80.032351,https://www.redfin.com/FL/Riviera-Beach/4000-N...,Emmy Buckman,Illustrated Properties LLC (Co,Owen Buckman,Illustrated Properties LLC (Co,Meike MacGregor,Compass Florida LLC,None,None,7,blue
7,PAST SALE,2025-06-13,Condo/Co-op,2800 S Ocean Blvd S Unit 19 C,Boca Raton,FL,33432.0,1635700.0,2.0,2.0,Ocean Towers Condo,1425.0,NaN,1973.0,NaN,1148.0,1262.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Beaches MLS,RX-11045437,N,Y,26.323476,-80.075019,https://www.redfin.com/FL/Boca-Raton/2800-S-Oc...,Andrew Pietrak,Luxury Real Estate Group LLC,None,None,Robert Goodman,LoKation,None,None,8,blue
8,PAST SALE,2025-

In [36]:
%store -r map_box_api_key

In [37]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=9,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [38]:
m.save('index.html')

## Data snagger

In [39]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [40]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-13,Condo/Co-op,1100 S Flagler Dr #1502,West Palm Beach,FL,33401.0,12543000.0,3.0,4.5,Bristol Condo,3815.0,NaN,2019.0,NaN,3288.0,8327.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-11029308,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Christian Angle,Christian Angle Real Estate,None,None,Julien Taylor IV,William Raveis Real Estate,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [41]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_06162025 


# CREATE TEMPLATE 

In [42]:
muni_set = set(df_top_ten['CITY'])

In [43]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [44]:
df_top_ten.reset_index(inplace=True,drop=True)

In [45]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [46]:
top_sale

'Bristol Condo at 1100 S Flagler Dr #1502 in West Palm Beach'

In [47]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-13,Condo/Co-op,1100 S Flagler Dr #1502,West Palm Beach,FL,33401.0,12543000.0,3.0,4.5,Bristol Condo,3815.0,NaN,2019.0,NaN,3288.0,8327.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-11029308,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Christian Angle,Christian Angle Real Estate,None,None,Julien Taylor IV,William Raveis Real Estate,None,None,1,orange,Bristol Condo at 1100 S Flagler Dr #1502 in We...
1,PAST SALE,2025-06-11,Condo/Co-op,220 SE Mizner Blvd #303,Boca Raton,FL,33432.0,2753000.0,2.0,3.0,Alina Residences,2701.0,180.0,2024.0,NaN,1019.0,2283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11091056,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Alejandro Salazar,Douglas Elliman,Non Non-Member,Non-Member Selling Office,None,None,2,blue,Alina Residences at 220 SE Mizner Blvd #303 in...
2,PAST SALE,2025-06-11,Condo/Co-op,184 Lake Dr Unit 1-201,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,Havn Yacht Club & Residences,2662.0,NaN,2008.0,NaN,939.0,3993.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Shores/18...,Beaches MLS,RX-11076265,N,Y,26.781054,-80.039562,https://www.redfin.com/FL/Palm-Beach-Shores/18...,Thomas Murray,Compass Florida LLC (PB),None,None,Thomas Murray,Compass Florida LLC (PB),None,None,3,blue,Havn Yacht Club & Residences at 184 Lake Dr Un...
3,PAST SALE,2025-06-13,Condo/Co-op,220 SE Mizner Blvd #102,Boca Raton,FL,33432.0,2039000.0,2.0,2.5,Alina Residences,1937.0,180.0,2024.0,NaN,1053.0,1625.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11090640,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Alejandro Salazar,Douglas Elliman,Non Agent,Non-Member Selling Office,None,None,4,blue,Alina Residences at 220 SE Mizner Blvd #102 in...
4,PAST SALE,2025-06-13,Condo/Co-op,200 E Palmetto Park Rd #409,Boca Raton,FL,33432.0,1875000.0,2.0,2.5,200 East Palmetto Park Condo,2018.0,2018.0,2009.0,NaN,929.0,2292.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Beaches MLS,RX-11030421,N,Y,26.350338,-80.082988,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Scott Eckert,RPE Realty,None,None,5,blue,200 East Palmetto Park Condo at 200 E Palmetto...
5,PAST SALE,2025-06-10,Condo/Co-op,170 N Ocean Blvd #307,Palm Beach,FL,33480.0,1690800.0,2.0,2.0,Ocean Towers North Condo,1395.0,NaN,1968.0,NaN,1212.0,2138.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Beaches MLS,RX-11013110,N,Y,26.721061,-80.035205,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Karen Lehrman Prayias,"Sotheby's Intl. Realty, Inc.",None,None,Karen Lehrman Prayias,"Sotheby's Intl. Realty, Inc.",None,None,6,blue,Ocean Towers North Condo at 170 N Ocean Blvd #...
6,PAST SALE,2025-06-09,Condo/Co-op,4000 N Ocean Dr #301,Singer Island,FL,33404.0,1650000.0,2.0,3.5,Martinique,2011.0,NaN,1986.0,NaN,820.0,1901.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4000-N...,Beaches MLS,RX-11053747,N,Y,26.795862,-80.032351,https://www.redfin.com/FL/Riviera-Beach/4000-N...,Emmy Buckman,Illustrated Properties LLC (Co,Owen Buckman,Illustrated Properties LLC (Co,Meike MacGregor,Compass Florida LLC,None,None,7,blue,Martinique at 4000 N Ocean Dr #301 in Singer I...
7,PAST SALE,2025-06-13,Condo/Co-op,2800 S Ocean Blvd S Unit 19 C,Boca Rato

In [50]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Bristol Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,200,000 to $12,543,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, West Palm Beach, Palm Beach, Boca Raton, Singer Island, Palm Beach Shores, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 92 condo sales totaling $55,024,048 million from June 6 to June 14. The previous week, brokers closed 91 condo sales totaling $53,926,699.

Last week’s units sold for an average of $598,087, higher than the $592,601 average pric

In [51]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [52]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [53]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-13,Condo/Co-op,1100 S Flagler Dr #1502,West Palm Beach,FL,33401.0,12543000.0,3.0,4.5,Bristol Condo,3815.0,NaN,2019.0,NaN,3288.0,8327.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-11029308,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Christian Angle,Christian Angle Real Estate,None,None,Julien Taylor IV,William Raveis Real Estate,None,None,1,orange,Bristol Condo at 1100 S Flagler Dr #1502 in We...
1,PAST SALE,2025-06-11,Condo/Co-op,220 SE Mizner Blvd #303,Boca Raton,FL,33432.0,2753000.0,2.0,3.0,Alina Residences,2701.0,180.0,2024.0,NaN,1019.0,2283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11091056,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Alejandro Salazar,Douglas Elliman,Non Non-Member,Non-Member Selling Office,None,None,2,blue,Alina Residences at 220 SE Mizner Blvd #303 in...
2,PAST SALE,2025-06-11,Condo/Co-op,184 Lake Dr Unit 1-201,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,Havn Yacht Club & Residences,2662.0,NaN,2008.0,NaN,939.0,3993.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Shores/18...,Beaches MLS,RX-11076265,N,Y,26.781054,-80.039562,https://www.redfin.com/FL/Palm-Beach-Shores/18...,Thomas Murray,Compass Florida LLC (PB),None,None,Thomas Murray,Compass Florida LLC (PB),None,None,3,blue,Havn Yacht Club & Residences at 184 Lake Dr Un...
3,PAST SALE,2025-06-13,Condo/Co-op,220 SE Mizner Blvd #102,Boca Raton,FL,33432.0,2039000.0,2.0,2.5,Alina Residences,1937.0,180.0,2024.0,NaN,1053.0,1625.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11090640,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Alejandro Salazar,Douglas Elliman,Non Agent,Non-Member Selling Office,None,None,4,blue,Alina Residences at 220 SE Mizner Blvd #102 in...
4,PAST SALE,2025-06-13,Condo/Co-op,200 E Palmetto Park Rd #409,Boca Raton,FL,33432.0,1875000.0,2.0,2.5,200 East Palmetto Park Condo,2018.0,2018.0,2009.0,NaN,929.0,2292.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Beaches MLS,RX-11030421,N,Y,26.350338,-80.082988,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Scott Eckert,RPE Realty,None,None,5,blue,200 East Palmetto Park Condo at 200 E Palmetto...
5,PAST SALE,2025-06-10,Condo/Co-op,170 N Ocean Blvd #307,Palm Beach,FL,33480.0,1690800.0,2.0,2.0,Ocean Towers North Condo,1395.0,NaN,1968.0,NaN,1212.0,2138.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Beaches MLS,RX-11013110,N,Y,26.721061,-80.035205,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Karen Lehrman Prayias,"Sotheby's Intl. Realty, Inc.",None,None,Karen Lehrman Prayias,"Sotheby's Intl. Realty, Inc.",None,None,6,blue,Ocean Towers North Condo at 170 N Ocean Blvd #...
6,PAST SALE,2025-06-09,Condo/Co-op,4000 N Ocean Dr #301,Singer Island,FL,33404.0,1650000.0,2.0,3.5,Martinique,2011.0,NaN,1986.0,NaN,820.0,1901.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4000-N...,Beaches MLS,RX-11053747,N,Y,26.795862,-80.032351,https://www.redfin.com/FL/Riviera-Beach/4000-N...,Emmy Buckman,Illustrated Properties LLC (Co,Owen Buckman,Illustrated Properties LLC (Co,Meike MacGregor,Compass Florida LLC,None,None,7,blue,Martinique at 4000 N Ocean Dr #301 in Singer I...
7,PAST SALE,2025-06-13,Condo/Co-op,2800 S Ocean Blvd S Unit 19 C,Boca Rato

In [55]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Bristol Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,200,000 to $12,543,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, West Palm Beach, Palm Beach, Boca Raton, Singer Island, Palm Beach Shores, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 92 condo sales totaling $55,024,048 million from June 6 to June 14. The previous week, brokers closed 91 condo sales totaling $53,926,699.

Last week’s units sold for an average of $598,087, higher than the $592,601 average pric

In [63]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/West-Palm-Beach/1100-S-Flagler-Dr-33401/unit-1502/home/168183440


In [64]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Boca-Raton/220-SE-Mizner-Blvd-33432/unit-303/home/195825379


In [65]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Boca-Raton/19980-Sawgrass-Ln-33434/unit-5104/home/42638716


In [66]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Boca-Raton/4301-N-Ocean-Blvd-33431/unit-201/home/42320754


In [67]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/West-Palm-Beach/1100-S-Flagler-Dr-33401/unit-1502/home/168183440


In [68]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Boca-Raton/4301-N-Ocean-Blvd-33431/unit-201/home/42320754


In [69]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-13,Condo/Co-op,1100 S Flagler Dr #1502,West Palm Beach,FL,33401.0,12543000.0,3.0,4.5,Bristol Condo,3815.0,NaN,2019.0,NaN,3288.0,8327.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1100...,Beaches MLS,RX-11029308,N,Y,26.703917,-80.049385,https://www.redfin.com/FL/West-Palm-Beach/1100...,Christian Angle,Christian Angle Real Estate,None,None,Julien Taylor IV,William Raveis Real Estate,None,None,1,orange,Bristol Condo at 1100 S Flagler Dr #1502 in We...
1,PAST SALE,2025-06-11,Condo/Co-op,220 SE Mizner Blvd #303,Boca Raton,FL,33432.0,2753000.0,2.0,3.0,Alina Residences,2701.0,180.0,2024.0,NaN,1019.0,2283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11091056,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Alejandro Salazar,Douglas Elliman,Non Non-Member,Non-Member Selling Office,None,None,2,blue,Alina Residences at 220 SE Mizner Blvd #303 in...
2,PAST SALE,2025-06-11,Condo/Co-op,184 Lake Dr Unit 1-201,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,Havn Yacht Club & Residences,2662.0,NaN,2008.0,NaN,939.0,3993.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Shores/18...,Beaches MLS,RX-11076265,N,Y,26.781054,-80.039562,https://www.redfin.com/FL/Palm-Beach-Shores/18...,Thomas Murray,Compass Florida LLC (PB),None,None,Thomas Murray,Compass Florida LLC (PB),None,None,3,blue,Havn Yacht Club & Residences at 184 Lake Dr Un...
3,PAST SALE,2025-06-13,Condo/Co-op,220 SE Mizner Blvd #102,Boca Raton,FL,33432.0,2039000.0,2.0,2.5,Alina Residences,1937.0,180.0,2024.0,NaN,1053.0,1625.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Beaches MLS,RX-11090640,N,Y,26.347042,-80.083200,https://www.redfin.com/FL/Boca-Raton/220-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,Alejandro Salazar,Douglas Elliman,Non Agent,Non-Member Selling Office,None,None,4,blue,Alina Residences at 220 SE Mizner Blvd #102 in...
4,PAST SALE,2025-06-13,Condo/Co-op,200 E Palmetto Park Rd #409,Boca Raton,FL,33432.0,1875000.0,2.0,2.5,200 East Palmetto Park Condo,2018.0,2018.0,2009.0,NaN,929.0,2292.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Beaches MLS,RX-11030421,N,Y,26.350338,-80.082988,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Scott Eckert,RPE Realty,None,None,5,blue,200 East Palmetto Park Condo at 200 E Palmetto...
5,PAST SALE,2025-06-10,Condo/Co-op,170 N Ocean Blvd #307,Palm Beach,FL,33480.0,1690800.0,2.0,2.0,Ocean Towers North Condo,1395.0,NaN,1968.0,NaN,1212.0,2138.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Beaches MLS,RX-11013110,N,Y,26.721061,-80.035205,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Karen Lehrman Prayias,"Sotheby's Intl. Realty, Inc.",None,None,Karen Lehrman Prayias,"Sotheby's Intl. Realty, Inc.",None,None,6,blue,Ocean Towers North Condo at 170 N Ocean Blvd #...
6,PAST SALE,2025-06-09,Condo/Co-op,4000 N Ocean Dr #301,Singer Island,FL,33404.0,1650000.0,2.0,3.5,Martinique,2011.0,NaN,1986.0,NaN,820.0,1901.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4000-N...,Beaches MLS,RX-11053747,N,Y,26.795862,-80.032351,https://www.redfin.com/FL/Riviera-Beach/4000-N...,Emmy Buckman,Illustrated Properties LLC (Co,Owen Buckman,Illustrated Properties LLC (Co,Meike MacGregor,Compass Florida LLC,None,None,7,blue,Martinique at 4000 N Ocean Dr #301 in Singer I...
7,PAST SALE,2025-06-13,Condo/Co-op,2800 S Ocean Blvd S Unit 19 C,Boca Rato

## Time on Market Calculator

In [72]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 5, 15) ## List (Earlier) date
date2 = datetime(2025, 6, 11) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

27


## Clean CSV for Datawrapper Chart

In [73]:
chart_df = df_top_ten

In [74]:
chart_df = chart_df.fillna(" ")

In [75]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [76]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [77]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [78]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [79]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [80]:
chart_df['PPSF'].astype(float)

0    3288.0
1    1019.0
2     939.0
3    1053.0
4     929.0
5    1212.0
6     820.0
7    1148.0
8     522.0
9     675.0
Name: PPSF, dtype: float64

In [81]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [82]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [83]:
csv_date_string = today.strftime("%m_%d_%Y")

In [84]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [85]:
chart_df

,SOLD DATE,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-06-13,1100 S Flagler Dr #1502 in West Palm Beach,"$12,543,000","3,815","$3,288",2019,Christian Angle Christian Angle Real Estate,Julien Taylor IV William Raveis Real Estate
1,2025-06-11,220 SE Mizner Blvd #303 in Boca Raton,"$2,753,000","2,701","$1,019",2024,Rochelle LeCavalier Douglas Elliman Alejandro ...,Non Non-Member Non-Member Selling Office
2,2025-06-11,184 Lake Dr Unit 1-201 in Palm Beach Shores,"$2,500,000","2,662",$939,2008,Thomas Murray Compass Florida LLC (PB),Thomas Murray Compass Florida LLC (PB)
3,2025-06-13,220 SE Mizner Blvd #102 in Boca Raton,"$2,039,000","1,937","$1,053",2024,Rochelle LeCavalier Douglas Elliman Alejandro ...,Non Agent Non-Member Selling Office
4,2025-06-13,200 E Palmetto Park Rd #409 in Boca Raton,"$1,875,000","2,018",$929,2009,Michael Wells Premier Estate Properties Inc Sc...,Scott Eckert RPE Realty
5,2025-06-10,170 N Ocean Blvd #307 in Palm Beach,"$1,690,800","1,395","$1,212",1968,"Karen Lehrman Prayias Sotheby's Intl. Realty, ...","Karen Lehrman Prayias Sotheby's Intl. Realty, ..."
6,2025-06-09,4000 N Ocean Dr #301 in Singer Island,"$1,650,000","2,011",$820,1986,Emmy Buckman Illustrated Properties LLC (Co Ow...,Meike MacGregor Compass Florida LLC
7,2025-06-13,2800 S Ocean Blvd S Unit 19 C in Boca Raton,"$1,635,700","1,425","$1,148",1973,Andrew Pietrak Luxury Real Estate Group LLC,Robert Goodman LoKation
8,2025-06-10,4301 N Ocean Blvd #201 in Boca Raton,"$1,387,500","2,660",$522,1981,Lisa Nanfra Compass Florida LLC Beth Aronson F...,Sian Matthew Coldwell Banker
9,2025-06-09,19980 Sawgrass Ln #5104 in Boca Raton,"$1,200,000","1,779",$675,1979,Abigail Wapner Abigail Wapner Realty Barbara O...,Ina Tuchman Boca West Realty LLC
